In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import keras 
from keras.utils import np_utils 
import seaborn as sns
sns.set()

In [ ]:
root = '../input/competitive-data-science-predict-future-sales/'
t_root = '../input/translated-data/'
d1 = pd.read_csv(t_root + 'categories.csv')
d2 = pd.read_csv(t_root + 'items.csv')
d3 = pd.read_csv(root + 'sales_train.csv')
d4 = pd.read_csv(t_root + 'shops.csv')
test = pd.read_csv(root + 'test.csv')

In [ ]:
d=list()
d.append(d1),d.append(d2),d.append(d3),d.append(d4),
for i in range(4):
    print(d[i].head())
    print('\n')

In [ ]:
d3.info()

Since the sales_train data i.e d3 has both shop_id & item_id and have no missing values it can be used as an primary Dataset.

In [ ]:
sales = pd.merge(d3,d2, on='item_id',how='left')
sales = pd.merge(sales,d1, on='category_id',how='left')
sales = pd.merge(sales,d4, on='shop_id',how='left')

In [ ]:
sales.head()

### Checking for missing Values 

In [ ]:
sales.describe()

### Since the Count of all the columns we are equal, we can conclude that we got no missing values.

### **checking for outliers**
###### *the columns that we gotta be extra careful are item_cnt_day,item_price*

In [ ]:
sns.boxplot(sales['item_price'])

In [ ]:
sns.boxplot(sales['item_cnt_day'])

In [ ]:
sns.jointplot(sales['item_cnt_day'],sales['item_price'])

So both the columns have a few extreme values, since there are only a few of them we can simply ignore them or just impute them 

In [ ]:
sales['item_price'].describe()

In [ ]:
sales['item_price'].max()

Let's Try to impute the values of item_price using the simple quantile method of dealing with outliers,
It assumes that our data is normally distributed 

In [ ]:
temp1 = sales.copy()

In [ ]:
q1 = temp1['item_price'].quantile(0.25)
q3 = temp1['item_price'].quantile(0.75)

In [ ]:
iqr = q3 - q1
iqr

In [ ]:
upper_lm = q3 + 1.5*iqr
lower_lm = 0
upper_lm,lower_lm


In [ ]:
def imputer(value):
    if value > upper_lm:
        return upper_lm
    if value < lower_lm:
        return lower_lm
    else:
        return value

In [ ]:
temp1['item_price'] = temp1['item_price'].apply(imputer)

In [ ]:
temp1['item_price'].describe()

In [ ]:
plt.hist(temp1['item_price'])

In [ ]:
sns.boxplot(temp1['item_price'])

Now let's do a quick sanity check and see if we use this method for treating outliers how well it's gonna perform on dataset.

In [ ]:
temp1['item_price'].max()

In [ ]:
x= sales[sales['item_price'] > 2124]

In [ ]:
x

As we can see this imputation by quantile method though makes the data normally distributed but makes us lose a lot of vital information and therefore can lead to a false predictions 

so let's see if we can ignore the outliers.

In [ ]:
sns.boxplot(sales['item_price'])

In [ ]:
## since the data got only one outliers and rest other data is in less 70000
sales.item_price.max()
x = sales[sales['item_price'] > 50000]
x

#### since the data got only three such values which are greater than 50000 let's see if we can ignore them
#### for this we can check out the test data and see if the test data have the following rows or not

In [ ]:
test.head()

In [ ]:
a= test[test['item_id']==6066]
b = test[test['item_id']==13199]
c = test[test['item_id']==11365]

In [ ]:
a.info()

In [ ]:
b.info()

In [ ]:
c.info()

Since the item_id with 11365 is present in the test data we can't ignore that value and hence the upper limit of data should be set to 60000 while the lower limit should be set to 0 since we can't have a -ve value as price. lol that would imply cashier paying you for buying the product.

In [ ]:
sales = sales[sales['item_price'] < 60000]
sales['item_price'].describe()

In [ ]:
sales.describe()

Now to check the items with price below 0 

In [ ]:
x = sales[sales['item_price'] < 0]
x

In [ ]:
y = test[test['item_id'] == 2973]
y

Since this item_id isn't present in the test dataset it's better to ignore this isntead of imputing it using 0.

In [ ]:
sales = sales[sales['item_price'] > 0]
sales.describe()

### Treating the outliers in the column item_cnt_day

Fortunately sometimes ignoring abnormally high value in a single columnn results in treating outliers or high values in other columns since the whole row gets deleted. But it's always better to crosscheck the same. 
#Safe_practices

In [ ]:
sales['item_cnt_day'].describe()

As we can see that we have a minimumm value of -22 which might be the number of that certain item re
returend back to the shop, this isn't a outlier but since we only want to predict the number of item being sold it's better to remove this info as it might be redundant and can cause problem at the time of final prediction. 

In [ ]:
returned_items = sales[sales['item_cnt_day'] < 0]
returned_items['item_cnt_day'].unique()

In [ ]:
returned_items

But since we there are around more than 7000 rows that have -ve values which implies the items that were returned back to the store on different dates. Just ignoring all of them would mean loosing a lot of imforamtion so it's better to make a copy of data before proceeding further.

We can generate new feature using this new data frame, like the frequency of a particular item being returned in a month so better train our model. 

In [ ]:
sales_copy = sales.copy()

In [ ]:
sales = sales[sales['item_cnt_day'] > 0]

In [ ]:
sales['item_cnt_day'].describe()

In [ ]:
sns.boxplot(sales['item_cnt_day'])

Since most of the data has the values below approx 1100, let's see what item is the outlier and if it's present in the test data as well.

In [ ]:
x = sales[sales['item_cnt_day'] >= 1000]
x

In [ ]:
y = test[test['item_id']==11373 ]
y

Since the item is available in the data it won't be a good choice to simply remove this,
since a very high value can sometimes alter the whole population mean and just because we have the evaluation metircs as RMSE which takes the mean squared error so keeping this value as it is might affect the whole model's accuracy and loss.
let's first let's make a copy of the sales data and in the copied data, 
we'll make two models 1st with the original data with the count of the item unchnaged & The 2nd model with the item count changed to 1000 since that is the upper limit.
futhermore the product with item_id 20949, Corporate package T-shirt 1C Interest white is also a single item but a pretty high value, it might affect the mean too, so in the copied data we can clip it's value to 700 as all the other values are below 669.

In [ ]:
sales_copy = sales.copy()

In [ ]:
sales_copy1 = sales_copy[sales_copy['item_cnt_day'] <= 1000]
sales_copy1['item_cnt_day'].describe()

In [ ]:
sales_copy2 = sales_copy.copy()

In [ ]:
sales_copy2['item_cnt_day'] = np.where(sales_copy2['item_cnt_day'] >=1000 , int(670),sales_copy2['item_cnt_day'])

In [ ]:
sales_copy2['item_cnt_day'].describe()

Basic LSTM model.

for LSTM model we'll need a sequence of timestamps to be fed to LSTM, since we already have date_block_num which hows the month, we can take up a sum of item_cnt.

For this model i'm gonna use the Sales_copy dataset which have the values 2169 included.

In [ ]:
sales_copy.head()

In [ ]:
sales_copy['item_id'].min()

So a LSTM model works with Time stamps or Sequences and since we already have date_num_blocks as the months number in a increasing order, we can use that as the timeseq varaible.

so the approach is goning to be pretty basic i'm gonna use the information provided in the train data about the item_id, shop_id merge it with the test data, since we need the final prediction on test data, i'm gonna use the first 33 months as input and try to get the prediction on of 34th month i.e october 2015, and compare our predictions with the values of 34th month that we already have. So we are going to use the 34th months values as label to our data.

</br> After this I'm gonna feed the value of months from 2-34 column since my LSTM would expect a imput of 33 values and try to predict the values for 35th month i.e november.

In [ ]:
lstm_dat = pd.pivot_table(sales_copy,index=['shop_id','item_id'],columns=['date_block_num'],values=['item_cnt_day'],fill_value=0,aggfunc=sum)

I had to pivot the table on both the shop_id & item_id cause every shop have same ids for every item and taking a sum of them without segregating them would result in the items_count getting summed over all the shops or shops getting summed by different item_id and since in the test dataset the items are segregated shop_wise and then item_wise.

In [ ]:
lstm_dat

In [ ]:
lstm_dat.reset_index(inplace=True)

In [ ]:
lstm_dat

Merging the data would result in the item_cnt getting mapped to the respective item_ids according to months and the redundant rows getting dropped since it'll be a left join.

In [ ]:
time_seq_dat1 = pd.merge(test,lstm_dat,on=['item_id','shop_id'],how='left')

In [ ]:
time_seq_dat1.fillna(0,inplace=True)
time_seq_dat1.head()

Now the working of a LSTM is pretty simple we give a sequence of the numbers in a increasing order of timestamp and we predict the next the value in the series. Since we won't need ID, Item_id, Shop_id 
we'll drop them

In [ ]:
time_seq_dat = time_seq_dat1.drop(columns=['ID','item_id','shop_id'])

In [ ]:
time_seq_dat.fillna(0,inplace=True)

Now we have a sequence of the item_cnt_days in a increasing order of time.

In [ ]:
time_seq_dat.head()

now we gotta make inputs and output variable, so for input we'll use the first 33 months, and use the lables of 34th month i.e october 2015 as output to be the labels or output variable.

--> for X_test we'll use the columns or timestamp 2-33 and try to predict the output for 35th i.e November 2015  


In [ ]:
time_seq_dat.shape

In [ ]:
x_train = np.reshape(time_seq_dat.values[:,:-1],newshape=(-1,33,1))
y_train = np.reshape(time_seq_dat.values[:,-1:],newshape=(-1,1,1))
x_train.shape,y_train.shape ##sanity check 

In [ ]:
x_test = np.reshape(time_seq_dat.values[:,1:],newshape=(-1,33,1))
x_test.shape

Now to make the final model
A bidirectional LSTM model we learn the input in both the directions forward and backwards. 

In [ ]:
import keras 
from keras.models import Sequential
from keras import layers as l 

model1 = Sequential()
model1.add(l.Bidirectional(l.LSTM(100,activation = 'relu',input_shape = (33,1))))
model1.add(l.Dense(1))

model1.compile(optimizer='adam', loss='mse',metrics = ['MSE'])

In [ ]:
x_train.shape

In [ ]:
x_train.shape[0]/2048

In [ ]:
history = model1.fit(x_train,y_train,batch_size = 2048 ,epochs = 50)

In [ ]:
sub1 = model1.predict(x_test,verbose=1)

In [ ]:
sales_copy['item_cnt_day'].max()

In [ ]:
submission = pd.DataFrame({'ID':test['ID'],'item_cnt_month':sub1.ravel()})

In [ ]:
submission

In [ ]:
submission.to_csv('sub14.csv',index = False)